In [1]:
#All Documents In Hand
#filled_form="D:\Mini projrct\filled_form.png"
#unfilled_form="D:\Mini projrct\unfilled_form.png"
#partialyFilled="D:\Mini projrct\partialyFilled.png"
#new_filled="D:\Mini projrct\New_filled.png"

In [2]:
#importing necessary Libriaries
import fitz
import cv2
import numpy as np

In [8]:
#miniproject Final Code
#Converting PDF to Image formate

def pdf_to_images(pdf_file):
    pdf_document = fitz.open(pdf_file)
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        image_matrix = fitz.Matrix(fitz.Identity)
        image = page.get_pixmap(matrix=image_matrix, alpha=False)
        image.save(f"page_{page_num + 1}.png")
    pdf_document.close()


#Handwritten_final_code

def count_template_matches_Handwritten(input_image_path, h_template_paths, min_threshold=0.6, max_threshold=0.8, min_matches=10):
    # Read the input image
    input_image = cv2.imread(input_image_path)

    # Convert the input image to grayscale
    input_gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Initialize a dictionary to store the average matching scores for each template
    template_scores_h = {}

    for template_path_hand in h_template_paths:
        # Read the template image and convert it to grayscale
        template_h = cv2.imread(template_path_hand, cv2.IMREAD_GRAYSCALE)

        # Get the width and height of the template image
        w, h = template_h.shape[::-1]

        # Perform template matching using cv2.matchTemplate with cv2.TM_CCOEFF_NORMED method
        result_h = cv2.matchTemplate(input_gray, template_h, cv2.TM_CCOEFF_NORMED)

        # Find the locations where the result exceeds the threshold
        loc = np.where((result_h >= min_threshold) & (result_h <= max_threshold))

        # Calculate the average matching score for the current template
        if len(loc[0]) > 0:  # Check if there are matches
            avg_score_h = np.mean(result_h[loc])
            template_scores_h[template_path_hand] = avg_score_h

            # Draw rectangles around the matched regions in the input image
            for pt in zip(*loc[::-1]):
                cv2.rectangle(input_image, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)

    return input_image, template_scores_h


#Signature is fiiled or not
def count_template_matches_signature(input_image_path, signature_template_paths, threshold_s=0.59):
    # Read the input image
    input_image = cv2.imread(input_image_path)

    # Convert the input image to grayscale
    input_gray_s = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Initialize a dictionary to store the number of matches for each template
    template_matches_s = {}

    for template_path_s in signature_template_paths:
        # Read the template image and convert it to grayscale
        template_s = cv2.imread(template_path_s, cv2.IMREAD_GRAYSCALE)

        # Get the width and height of the template image
        w, h = template_s.shape[::-1]

        # Perform template matching using cv2.matchTemplate with cv2.TM_CCOEFF_NORMED method
        result_s = cv2.matchTemplate(input_gray_s, template_s, cv2.TM_CCOEFF_NORMED)

        # Find the locations where the result exceeds the threshold
        loc_s = np.where(result_s >= threshold_s)

        # Count the number of matches found for the current template
        num_matches_s = len(loc_s[0])

        # Store the number of matches for the current template in the dictionary
        template_matches_s[template_path_s] = num_matches_s
    return input_image, template_matches_s




#seal checking
def is_circular_shape(contour):
    # Calculate the contour area
    contour_area = cv2.contourArea(contour)

    # Calculate the minimum enclosing circle for the contour
    (x, y), radius = cv2.minEnclosingCircle(contour)

    # Calculate the circularity as the ratio of contour area to the enclosing circle area
    circularity = contour_area / (np.pi * radius * radius)

    # Check if the contour area and circularity fall within the criteria for a circular shape
    return contour_area > 100 and circularity > 0.6

def is_oval_shape(contour):
    # Calculate the contour area and the bounding box of the contour
    contour_area = cv2.contourArea(contour)
    x, y, w, h = cv2.boundingRect(contour)

    # Calculate the aspect ratio of the bounding box
    aspect_ratio = w / float(h)

    # Check if the contour area and aspect ratio fall within the criteria for an oval shape
    return contour_area > 100 and 0.69 <= aspect_ratio <= 1.3

def count_template_matches(input_image_path, seal_template_paths):
    # Read the input image as grayscale
    input_image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

    # Initialize variables to count matches
    circular_matches = 0
    oval_matches = 0

    for template_path_seal in seal_template_paths:
        # Read the template image as grayscale
        template_seal = cv2.imread(template_path_seal, cv2.IMREAD_GRAYSCALE)

        # Perform template matching
        result_seal = cv2.matchTemplate(input_image, template_seal, cv2.TM_CCOEFF_NORMED)
        seal_threshold = 0.7  # Set a threshold for the matches (adjust as needed)

        # Get the locations of matches above the threshold
        loc_seal = np.where(result_seal >= seal_threshold)

        # Find contours in the input image
        contours, _ = cv2.findContours(input_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Check if the matched regions are circular or oval
        for pt in zip(*loc_seal[::-1]):
            # Convert the points to integer values
            pt = (int(pt[1]), int(pt[0]))

            # Find the contour that contains the matched point
            matched_contour = None
            for contour in contours:
                if cv2.pointPolygonTest(contour, pt, False) >= 0:
                    matched_contour = contour
                    break

            if matched_contour is not None:
                if is_circular_shape(matched_contour):
                    circular_matches += 1
                elif is_oval_shape(matched_contour):
                    oval_matches += 1

    return circular_matches, oval_matches


#calling all functions

#converting pdf to image

pdf_to_images(r"D:\Mini projrct\experiance_certificate___forest_boat_driver (1).pdf")

# inputing necessary data
input_image_path = 'page_1.png'
h_template_paths = [r"D:\Mini projrct\template2.png", r"D:\Mini projrct\template4.png"]
signature_template_paths = [r"D:\Mini projrct\template_sig.png", r"D:\Mini projrct\template_sig2.png"]
seal_template_paths = [r"D:\Mini projrct\seal2.png",r"D:\Mini projrct\seal3.png"]
min_threshold = 0.7
max_threshold = 0.9
min_matches = 5
threshold_sign = 0.59

# Calling Handwritten text checking function

result_image_h, template_scores_handwritten = count_template_matches_Handwritten(input_image_path, h_template_paths, min_threshold, max_threshold)

# Filter out 'nan' scores from the template_scores dictionary
template_scores_handwritten = {k: v for k, v in template_scores_handwritten.items() if not np.isnan(v)}

# Print the average matching scores for each template
i=0
for template_path, avg_score in template_scores_handwritten.items():
    avg_score="{:.2f}".format(avg_score)
    print(f"Average matching score for template'{i+1}': {avg_score}")
    i+=1

# Check if the form is filled or not
total_templates_h = len(h_template_paths)
num_matches_h = len(template_scores_handwritten)

if num_matches_h == 0:
    print("The form is completely filled with handwritten text.")
elif num_matches_h >= total_templates_h:
    print("The form is completely unfilled with handwritten text.")
else:
    print("The form is partially filled with handwritten text.")


#calling signature checking function

result_image_signature, template_matches_sign = count_template_matches_signature(input_image_path, signature_template_paths, threshold_sign)

# Print the number of matches for each template
#for template_path, num_matches in template_matches.items():
    #print()

# Check if the form is filled or not
total_signature_matches = sum(template_matches_sign.values())
if total_signature_matches > 0:
    print("The form is not filled with sign.")
else:
    print("The form is filled with sign.")


#Calling Seal checking function
circular_matches, oval_matches = count_template_matches(input_image_path, seal_template_paths)
print()
# Printing the result

if circular_matches > 0 or oval_matches > 0:
    print("The form is filled with seals.")
else:
    print("The form is not filled with any seals.")

Average matching score for template'1': 0.81
Average matching score for template'2': 0.89
The form is completely unfilled with handwritten text.

The form is not filled with sign.

The form is not filled with any seals.
